In [1]:
import pandas as pd
import numpy as np
import random
import warnings
from sklearn.model_selection import train_test_split,learning_curve
from sklearn import metrics, ensemble
import matplotlib.pyplot as plt
import statistics
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

/home/vladimir/.local/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
wines = pd.read_csv("winequality-red.csv",sep = ';')
wines.shape

(1599, 12)

In [4]:
wines = wines.dropna()
wines['quality'] = wines['quality'].replace(to_replace=3, value=4).replace(8, 7).replace(4, 5)

In [5]:
wines.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
X = wines.drop(columns = ['quality'])
y = wines['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## GradientBoostingClassifier

In [7]:
param_model = GradientBoostingClassifier()
parameters = {
    "n_estimators":[50,100,500,], # Количество итераций бустинга
    "max_depth":[1,3,5], # Максимальное количество узлов в деревьях
    "learning_rate":[0.01,0.05,0.1] # Темп обучения
}

In [8]:
from sklearn.model_selection import RandomizedSearchCV
cv = RandomizedSearchCV(estimator = param_model, param_distributions = parameters, 
                                n_iter = 100, cv = 5, verbose = 2, 
                                random_state = 420, n_jobs = -1)
cv.fit(X_train,y_train)

D:\Users\Roman\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 27 is smaller than n_iter=100. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 27 candidates, totalling 135 fits


RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [1, 3, 5],
                                        'n_estimators': [50, 100, 500]},
                   random_state=420, verbose=2)

In [9]:
cv.best_params_

{'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.05}

In [7]:
model_final = GradientBoostingClassifier(loss ='exponential',
                                         learning_rate = 0.1, 
                                         max_depth = 5,
                                         n_estimators = 100)
model_final.fit(X_train, y_train)
score=cross_val_score(model_final,X_train,y_train,scoring='accuracy',cv=5)
tr_acc = []
tr_acc.append(statistics.mean(score))
print('CV-accuracy: ',statistics.mean(score))

ValueError: ExponentialLoss requires 2 classes; got 3 class(es)

In [11]:
y_pred = model_final.predict(X_test)
predictions = [round(value) for value in y_pred]
conf_mat = metrics.confusion_matrix(y_test, predictions)
print(conf_mat)
accuracy = accuracy_score(y_test, predictions)
test_acc = []
test_acc.append(accuracy)
print("Test Accuracy: %.2f" % (accuracy))

[[125  29   3]
 [ 34  75   9]
 [  5   9  31]]
Test Accuracy: 0.72


## AdaBoost

In [12]:
from sklearn.ensemble import AdaBoostClassifier

In [13]:
abc = AdaBoostClassifier()
abc.fit(X_train, y_train)
score=cross_val_score(abc,X_train,y_train,scoring='accuracy',cv=5)

print('CV-accuracy: ',statistics.mean(score))

CV-accuracy:  0.6152910539215686


In [14]:
y_pred = abc.predict(X_test)
predictions = [round(value) for value in y_pred]
conf_mat = metrics.confusion_matrix(y_test, predictions)
print(conf_mat)
accuracy = accuracy_score(y_test, predictions)

print("Test Accuracy: %.2f" % (accuracy))

[[105  50   2]
 [ 44  64  10]
 [  4  24  17]]
Test Accuracy: 0.58


In [15]:
ada_parameters = {
    "n_estimators":[50,100,500,], # Количество итераций 
    "learning_rate":[0.01,0.05,0.1] # Темп обучения
}
param_ada = AdaBoostClassifier()
ada_cv = RandomizedSearchCV(estimator = param_model, param_distributions = ada_parameters, 
                                n_iter = 100, cv = 5, verbose = 2, 
                                random_state = 420, n_jobs = -1)
ada_cv.fit(X_train,y_train)
ada_cv.best_params_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


D:\Users\Roman\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 9 is smaller than n_iter=100. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'n_estimators': 500, 'learning_rate': 0.05}

In [16]:
abc_final = AdaBoostClassifier(n_estimators=500,
                         learning_rate=0.05)
abc_final.fit(X_train, y_train)
score=cross_val_score(abc_final,X_train,y_train,scoring='accuracy',cv=5)
tr_acc.append(statistics.mean(score))
print('CV-accuracy: ',statistics.mean(score))

CV-accuracy:  0.6457843137254902


In [17]:
y_pred = abc_final.predict(X_test)
predictions = [round(value) for value in y_pred]
conf_mat = metrics.confusion_matrix(y_test, predictions)
print(conf_mat)
accuracy = accuracy_score(y_test, predictions)
test_acc.append(accuracy)
print("Test Accuracy: %.2f" % (accuracy))

[[112  45   0]
 [ 42  73   3]
 [  5  24  16]]
Test Accuracy: 0.63


## XGBoost

In [18]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [19]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
score=cross_val_score(model,X_train,y_train,scoring='accuracy',cv=5)
print('CV-accuracy: ',statistics.mean(score))

CV-accuracy:  0.7177267156862746


In [20]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
conf_mat = metrics.confusion_matrix(y_test, predictions)
print(conf_mat)
accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy: %.2f" % (accuracy))

[[122  32   3]
 [ 31  80   7]
 [  4   9  32]]
Test Accuracy: 0.73


In [21]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [22]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1), # The maximum depth of a tree
        'gamma': hp.uniform ('gamma', 1,9), # Gamma specifies the minimum loss reduction required to make a split
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1), # L1 regularization term on weights
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),# L2 regularization term on weights
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1), # the subsample ratio of columns when constructing each tree.
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1), # It defines the minimum sum of weights of all observations required in a child.
        'n_estimators': 180,
        'seed': 0
    }

In [23]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [24]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                  

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warn

SCORE:                                                                                                                 
0.509375                                                                                                               
SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.528125                                                                                                               
SCORE:                                  

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warn

SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                  

...

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warn

SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.490625                                                                                                               
 96%|███████████████████████████████████

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warn

SCORE:                                                                                                                 
0.490625                                                                                                               
SCORE:                                                                                                                 
0.5375                                                                                                                 
SCORE:                                                                                                                 
0.490625                                                                                                               
100%|████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 17.70trial/s, best loss: -0.55625]


D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warn

In [25]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'colsample_bytree': 0.7751985887684334, 'gamma': 2.7464376695108363, 'max_depth': 16.0, 'min_child_weight': 8.0, 'reg_alpha': 67.0, 'reg_lambda': 0.4747515168268933}


In [26]:
xgb_model_final = xgb.XGBClassifier(best_hyperparams)
xgb_model_final.fit(X_train, y_train)
score=cross_val_score(xgb_model_final,X_train,y_train,scoring='accuracy',cv=5)
tr_acc.append(statistics.mean(score))
print('CV-accuracy: ',statistics.mean(score))

D:\Users\Roman\anaconda3\lib\site-packages\xgboost\core.py:568: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


CV-accuracy:  0.7177267156862746


In [27]:
y_pred = xgb_model_final.predict(X_test)
predictions = [round(value) for value in y_pred]
conf_mat = metrics.confusion_matrix(y_test, predictions)
print(conf_mat)
accuracy = accuracy_score(y_test, predictions)
test_acc.append(accuracy)
print("Test Accuracy: %.2f" % (accuracy))

[[122  32   3]
 [ 31  80   7]
 [  4   9  32]]
Test Accuracy: 0.73


In [28]:
tr_acc

[0.7208609068627451, 0.6457843137254902, 0.7177267156862746]

In [29]:
test_acc

[0.721875, 0.628125, 0.73125]

In [30]:
acc_df = pd.DataFrame(list(zip(tr_acc, test_acc)),
               columns =['Train accuracy', 'Test accuracy'])
acc_df.index = ['GradientBoostingClassifier', 'AdaBoost', 'XGBoost']
acc_df.transpose()

,GradientBoostingClassifier,AdaBoost,XGBoost
Train accuracy,0.720861,0.645784,0.717727
Test accuracy,0.721875,0.628125,0.731250
